## 카드 승인 내역 엑셀 처리
1단계로 카드승인내역에서 불필요한 라인, 불필요한 컬럼(가뱅점번호, 단말기번호) 삭제하기

In [3]:

# KICC 신용거래내역 엑셀 파일 테이터 처리

import pandas as pnds
import pickle
import datetime

# 1. 엑셀 파일을 읽어서 DataFrame 생성
filename = './downdata/20220630/신용거래내역조회 (11).xlsx'                                       # KICC 신용거래내역 엑셀파일
card_history_df = pnds.read_excel(filename, header=0, thousands = ',',          # 금액 천단위 구분자 ',' 고려
                                  dtype={'거래고유번호': str, '금액': 'int64'}) # 필요 컬럼의 데이터 타입 고정

# 2. 데이터 전처리
# 2-1. '거래일시' 컬럼명 수정
card_history_df.rename(columns={'거래일시▼': 'date'}, inplace=True)     # '거래일시' 컬럼명 'date'로 수정 => 다른 dataframe과 통일

# # 데이터 타입 변경
# card_history_df['금액'] = card_history_df['금액'].astype('int64', errors='ignore')      # '금액' : int64

# '거래고유번호' 컬럼에 있는 내용중에 NaN인 행 제거
card_history_df = card_history_df.dropna(subset=['거래고유번호'])

# 2-2. '승인번호' 중복 라인제거를 위한 중복 정보 추가
dup_series = card_history_df.duplicated(['승인번호'], keep=False)           # '승인번호' 중복 여부 Bool series 리턴, 중복되면 모두 True
card_history_df = pnds.concat([card_history_df, dup_series], axis=1)        # 결과를 dataframe column 으로 추가
card_history_df.rename(columns={0: 'dup'}, inplace=True)                    # 추가된 중복 컬럼이름을 'dup'로 변경

# 2-3. 중복 라인들을 제외하면서, 동시에 필요 컬럼들만 추출
card_history_df = card_history_df.loc[card_history_df['dup'] == False][['거래고유번호', '승인구분', 'date', '카드번호', '발급카드사', '매입카드사', '금액']]

# 2-4. date 기준 Sorting
card_history_df = card_history_df.sort_values(by=['date'])

# 2-5. date 포맷 변경: '%Y-%m-%d'
card_history_df['date'] = card_history_df['date'].map(lambda str_data: str_data.split()[0], na_action='ignore') # 날짜 포맷 : '%Y-%m-%d'
# 다른 방식으로서, 데이터타입을 datetime으로하고 date포맷을 변경하는 방법
# opera_df['TRX_DATE'] = opera_df['TRX_DATE'].astype('datetime64[ns]', errors='ignore')    # date 포맷 변경을 위해서 datetimee 형으로 변환
# opera_df['TRX_DATE'] = opera_df['TRX_DATE'].dt.strftime('%Y-%m-%d')     # 연산을 통해 포맷 변경 => 반환 타입은 일반 객체로 변경됨

# card_history_df

# DataFrame 저장
target_date = (datetime.datetime.now() - datetime.timedelta(1)).strftime('%Y%m%d')  # 어제날짜의 폴더 이름
base_dir = 'C:\\work\\pycard\\dtdata\\' + target_date
now_date = card_history_df.iloc[1]['date']      # 정렬된 상태에서의 첫번쨰 인덱스를 선택 (정렬 이전의 지료에서 1번 인덱스는 다음날의 자료일 가능성이 있음)
file_name = '/dt_kicc_history_' + now_date
data_dir = base_dir + file_name
with open(data_dir, "wb") as file:
    pickle.dump(card_history_df, file)

c:\ProgramData\Anaconda3\envs\main\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,거래고유번호,승인구분,date,단말기번호,카드번호,카드구분,발급카드사,매입카드사,가맹점번호,금액,...,승인번호,키인,원승인일자,입금예정일,승인결과,전자서명,수수료,입금예정금액,취소여부,취소일자
0,010133088669,승인,2022-07-01 01:37:32,108033,5395-9388-3386-6877,신용,해외마스타,하나구외환,960439198,197472,...,767610,Y,- -,- -,정상,Y,0,0,N,- -
1,010111533250,승인,2022-07-01 01:36:20,108033,5567-1714-4427-0162,신용,해외마스타,하나구외환,960439198,55440,...,99178,Y,- -,- -,정상,Y,0,0,N,- -
2,010121532431,승인,2022-07-01 01:35:13,108033,5557-9777-0340-1765,신용,해외마스타,하나구외환,960439198,53460,...,882674,Y,- -,- -,정상,Y,0,0,N,- -
3,010133087601,승인,2022-07-01 01:33:14,108033,5567-1728-4890-5270,신용,해외마스타,하나구외환,960439198,49896,...,76061,Y,- -,- -,정상,N,0,0,N,- -
4,010111529994,승인,2022-07-01 01:32:07,108033,5557-9776-7817-0130,신용,해외마스타,하나구외환,960439198,48114,...,884100,Y,- -,- -,정상,N,0,0,N,- -
5,302321362623,승인,2022-06-30 23:38:35,108033,4902-2082-2856-3318,신용,우리카드,비씨카드,777198403,69300,...,55903578,N,- -,- -,정상,Y,0,0,N,- -
6,302233003806,승인,2022-06-30 22:51:16,108033,9440-3235-3369-6618,체크,부산비씨체크,비씨카드,777198403,69300,...,55686441,N,- -,- -,정상,Y,0,0,N,- -
7,302221144382,승인,2022-06-30 22:16:42,108033,5549-5901-3668-3072,신용,KB플래티늄카드,국민카드,60757348,118800,...,30030721,N,- -,- -,정상,Y,0,0,N,- -
8,301933542186,승인,2022-06-30 19:05:56,108033,5587-4901-0177-1775,신용,삼성마스타법인,삼성카드,138060838,118800,...,6928834,N,- -,- -,정상,Y,0,0,N,- -
9,301833521219,승인,2022-06-30 18:59:17,108033,5365-1018-2464-7394,체크,카카오뱅크체크,국민카드,60757348,44000,...,24006365,N,- -,- -,정상,N,0,0,N,- -
